In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# We'll use the following variables:

cols_to_use = [
    "OverallQual",
    "TotalBsmtSF",
    "1stFlrSF",
    "GrLivArea",
    "WoodDeckSF",
    "BsmtUnfSF",
    "LotFrontage",
    "MasVnrArea",
    "GarageYrBlt",
    "BsmtQual",
    "FireplaceQu",
    "SalePrice",
]

In [3]:
# Load the House Prices dataset.

data = pd.read_csv("/content/drive/MyDrive/Feature Engineering/Datasets/houseprice.csv", usecols=cols_to_use)

data.head()

,LotFrontage,OverallQual,MasVnrArea,BsmtQual,BsmtUnfSF,TotalBsmtSF,1stFlrSF,GrLivArea,FireplaceQu,GarageYrBlt,WoodDeckSF,SalePrice
0,65.0,7,196.0,Gd,150,856,856,1710,NaN,2003.0,0,208500
1,80.0,6,0.0,Gd,284,1262,1262,1262,TA,1976.0,298,181500
2,68.0,7,162.0,Gd,434,920,920,1786,TA,2001.0,0,223500
3,60.0,7,0.0,TA,540,756,961,1717,Gd,1998.0,0,140000
4,84.0,8,350.0,Gd,490,1145,1145,2198,TA,2000.0,192,250000


In [4]:
# Let's separate into training and testing sets.

X_train, X_test, y_train, y_test = train_test_split(
    data.drop("SalePrice", axis=1),
    data["SalePrice"],
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((1022, 11), (438, 11))

In [5]:
# Capture numerical variables

vars_num = list(X_train.select_dtypes(include="number").columns)

vars_num

['LotFrontage',
 'OverallQual',
 'MasVnrArea',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'WoodDeckSF']

In [6]:
# Capture categorical variables

vars_cat = list(X_train.select_dtypes(exclude="number").columns)

vars_cat

['BsmtQual', 'FireplaceQu']

In [7]:
# Create the imputation dictionary

# median imputation for numeric
imputation_dict = X_train[vars_num].median().to_dict()

# add mode imputation for categoric
imputation_dict.update(X_train[vars_cat].mode().iloc[0].to_dict())

# show final dictionary
imputation_dict

{'LotFrontage': 69.0,
 'OverallQual': 6.0,
 'MasVnrArea': 0.0,
 'BsmtUnfSF': 486.5,
 'TotalBsmtSF': 992.0,
 '1stFlrSF': 1095.0,
 'GrLivArea': 1479.0,
 'GarageYrBlt': 1979.0,
 'WoodDeckSF': 0.0,
 'BsmtQual': 'TA',
 'FireplaceQu': 'Gd'}

In [8]:
# Calculate missing indicators: train set

X_train.isna().astype(int)

,LotFrontage,OverallQual,MasVnrArea,BsmtQual,BsmtUnfSF,TotalBsmtSF,1stFlrSF,GrLivArea,FireplaceQu,GarageYrBlt,WoodDeckSF
64,1,0,0,0,0,0,0,0,1,0,0
682,1,0,0,0,0,0,0,0,0,0,0
960,0,0,0,0,0,0,0,0,1,1,0
1384,0,0,0,0,0,0,0,0,1,0,0
1100,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
763,0,0,0,0,0,0,0,0,0,0,0
835,0,0,0,0,0,0,0,0,1,0,0
1216,0,0,0,1,0,0,0,0,1,0,0
559,1,0,0,0,0,0,0,0,0,0,0


In [9]:
# create indicator names

indicators = [f"{var}_na" for var in X_train.columns]

indicators

['LotFrontage_na',
 'OverallQual_na',
 'MasVnrArea_na',
 'BsmtQual_na',
 'BsmtUnfSF_na',
 'TotalBsmtSF_na',
 '1stFlrSF_na',
 'GrLivArea_na',
 'FireplaceQu_na',
 'GarageYrBlt_na',
 'WoodDeckSF_na']

In [10]:
# Add missing indicators to train set

X_train[indicators] = X_train.isna().astype(int)

X_train.head()

,LotFrontage,OverallQual,MasVnrArea,BsmtQual,BsmtUnfSF,TotalBsmtSF,1stFlrSF,GrLivArea,FireplaceQu,GarageYrBlt,...,OverallQual_na,MasVnrArea_na,BsmtQual_na,BsmtUnfSF_na,TotalBsmtSF_na,1stFlrSF_na,GrLivArea_na,FireplaceQu_na,GarageYrBlt_na,WoodDeckSF_na
64,NaN,7,573.0,Gd,318,1057,1057,2034,NaN,1998.0,...,0,0,0,0,0,0,0,1,0,0
682,NaN,6,0.0,Gd,288,1291,1291,1291,Gd,1996.0,...,0,0,0,0,0,0,0,0,0,0
960,50.0,5,0.0,TA,162,858,858,858,NaN,NaN,...,0,0,0,0,0,0,0,1,1,0
1384,60.0,6,0.0,TA,356,560,698,1258,NaN,1939.0,...,0,0,0,0,0,0,0,1,0,0
1100,60.0,2,0.0,TA,0,290,438,438,NaN,1930.0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
# Add indicators to test set

X_test[indicators] = X_test.isna().astype(int)

In [12]:
# After adding indicators, we can replace missing data
# in the original variables

X_train.fillna(imputation_dict, inplace=True)
X_test.fillna(imputation_dict, inplace=True)

In [13]:
# Corroborate replacement

X_train.isnull().sum()

LotFrontage       0
OverallQual       0
MasVnrArea        0
BsmtQual          0
BsmtUnfSF         0
TotalBsmtSF       0
1stFlrSF          0
GrLivArea         0
FireplaceQu       0
GarageYrBlt       0
WoodDeckSF        0
LotFrontage_na    0
OverallQual_na    0
MasVnrArea_na     0
BsmtQual_na       0
BsmtUnfSF_na      0
TotalBsmtSF_na    0
1stFlrSF_na       0
GrLivArea_na      0
FireplaceQu_na    0
GarageYrBlt_na    0
WoodDeckSF_na     0
dtype: int64

In [14]:
# Corroborate replacement

X_test.isnull().sum()

LotFrontage       0
OverallQual       0
MasVnrArea        0
BsmtQual          0
BsmtUnfSF         0
TotalBsmtSF       0
1stFlrSF          0
GrLivArea         0
FireplaceQu       0
GarageYrBlt       0
WoodDeckSF        0
LotFrontage_na    0
OverallQual_na    0
MasVnrArea_na     0
BsmtQual_na       0
BsmtUnfSF_na      0
TotalBsmtSF_na    0
1stFlrSF_na       0
GrLivArea_na      0
FireplaceQu_na    0
GarageYrBlt_na    0
WoodDeckSF_na     0
dtype: int64